# Geospatial Analysis on Starbucks data

In [1]:
import math
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode            
import folium 
from folium import Marker
from folium.plugins import MarkerCluster

In [2]:
from shapely.geometry import Point

In [3]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')



### 1) Geocode the missing locations.

 create a DataFrame `starbucks` containing Starbucks locations in the state of California.

In [4]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("data/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [5]:
starbucks.count()

Store Number    2821
Store Name      2821
Address         2821
City            2821
Longitude       2816
Latitude        2816
dtype: int64

Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

In [6]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


### to fill in these values with the OpenStreetMap Nominatim geocoder.

In [7]:
def my_geocoder(row):
    point = geocode(row, provider='nominatim').geometry[0]
    return pd.Series({'Longitude': point.x, 'Latitude': point.y})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

### 2) View Berkeley locations.

Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

In [8]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

#  Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Show the map
embed_map(m_2, 'output/geocodedLocationMap.html')

### 3) Consolidate your data.

to load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

In [9]:
CA_counties = gpd.read_file("data/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


Next, there are three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

In [10]:
CA_pop = pd.read_csv("data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("data/CA_county_median_age.csv", index_col="GEOID")

to join the `CA_counties` GeoDataFrame with `CA_pop`, `CA_high_earners`, and `CA_median_age`.

Name the resultant GeoDataFrame `CA_stats`, and it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".  Also, the CRS is set to `{'init': 'epsg:4326'}`.

In [11]:
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")


In [12]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

In [13]:
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


### 4) Which counties look promising?

Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `CA_stats` GeoDataFrame.  In particular, you should select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).



In [14]:
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]


In [15]:
# starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=[Point(x, y) for x, y in zip(starbucks.Longitude, starbucks.Latitude)])
starbucks_gdf.crs = {'init': 'epsg:4326'}

/Users/krishnakhadka/Library/Python/3.7/lib/python/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [16]:
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)
num_stores

/Users/krishnakhadka/Library/Python/3.7/lib/python/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


1043

### 5) Visualize the store locations.

Create a map that shows the locations of the stores that you identified in the previous question.

In [17]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Show the map
embed_map(m_6, 'output/finalMap.html')

/Users/krishnakhadka/Library/Python/3.7/lib/python/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


### 6) Now let's make the criteria more reliable and based on data.

From the above summary it is clear that there are 3 core variables (population, high_earners and median age) which can give a direct impact on the performance of stores in a particular county and one derived variable (density)

First step is to normalise the four columns. we will write a normalization function to do so. 

This gives an advantage over the previous methods as follows:
- You can pick and choose the top number of counties to look for the best store clusters on score.
- configure the weights to tune the output based on visualization
- entirely based on the data and logical weights and not complex filters

In [18]:
def normalizer(col):
    max_val = col.max()
    min_val = col.min()
    return (col - min_val)/(max_val-min_val)


In [19]:
CA_stats['population_norm'] = normalizer(CA_stats.population)
CA_stats['high_earners_norm'] = normalizer(CA_stats.high_earners)
CA_stats['median_age_norm'] = normalizer(CA_stats.median_age)
CA_stats['density_norm'] = normalizer(CA_stats.density)

In [20]:
#calculating the score by giving appropriate weightage to each of the normalized variable

def scoring(pop_wt,he_wt,age_wt,den_wt):
    score = CA_stats.population * pop_wt + CA_stats.high_earners * he_wt + CA_stats.median_age * age_wt +CA_stats.density * den_wt
    min_val = score.min()
    max_val = score.max()
    return (score - min_val)/(max_val-min_val)

# pop_wt = 5, population should not skew the overall selection of sites
# he_wt = 10, this is an important parameter to choose a region over the other
# age_wt = -3 more age tends to decrease the popularity
# den_wt = 7 denser the region better it is

CA_stats['final_score'] = scoring(5,10,-3,7)


In [21]:
sel_counties_based_on_score = CA_stats.sort_values('final_score',ascending = False)[0:5].copy()

In [22]:
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties_based_on_score)
num_stores = len(locations_of_interest)
num_stores

/Users/krishnakhadka/Library/Python/3.7/lib/python/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


1556

In [23]:
# Create a base map
base_map = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
top_locations = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties_based_on_score)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        top_locations.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

base_map.add_child(top_locations)

# Show the map
embed_map(base_map, 'output/finalMap_basedOnScore.html')

/Users/krishnakhadka/Library/Python/3.7/lib/python/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)
